# Agent Based Model: Main Notebook

In [32]:
import numpy.random as rnd
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt
import random

#### Workflow Notes 
* Draw Students with values within the four dimensions 
* Function that creates a study group (draw 4 students) -> Measure homogeniety 
* Task 


## Importing The Study Groups that are to be Tested

In [33]:
homogenous = pd.read_csv("Conditions/Homogenous.csv")
fiftyfifty = pd.read_csv("Conditions/FiftyFifty.csv")
heterogenous = pd.read_csv("Conditions/Heterogenous.csv")


for i in (range(len(homogenous.index))):
    print(list(homogenous.iloc[i][1:5]))


['ESFJ', 'ESFJ', 'ESFJ', 'ESFJ']
['ESTJ', 'ESTJ', 'ESTJ', 'ESTJ']
['ISFJ', 'ISFJ', 'ISFJ', 'ISFJ']
['ISTJ', 'ISTJ', 'ISTJ', 'ISTJ']
['ENFP', 'ENFP', 'ENFP', 'ENFP']
['ENFJ', 'ENFJ', 'ENFJ', 'ENFJ']
['INFP', 'INFP', 'INFP', 'INFP']
['INFJ', 'INFJ', 'INFJ', 'INFJ']
['ENTP', 'ENTP', 'ENTP', 'ENTP']
['ENTJ', 'ENTJ', 'ENTJ', 'ENTJ']
['INTP', 'INTP', 'INTP', 'INTP']
['INTJ', 'INTJ', 'INTJ', 'INTJ']
['ESFP', 'ESFP', 'ESFP', 'ESFP']
['ESTP', 'ESTP', 'ESTP', 'ESTP']
['ISFP', 'ISFP', 'ISFP', 'ISFP']
['ISTP', 'ISTP', 'ISTP', 'ISTP']


## Creating Study Groups

#### Practical Functions

In [34]:
def n_sampler(size):
    '''
    sample a number of random numbers from the beta distribution
    '''
    max_vals = []
    min_vals = []

    beta_dist = rnd.beta(1.5, 1.5, size)

    for i in range(size):
        if beta_dist[i] >= 0.5:
            max_vals.append(beta_dist[i])
        else:
            min_vals.append(beta_dist[i])
    
    return max_vals, min_vals

max_vals, min_vals = n_sampler(1000000)

In [35]:
def data_collect(studygroup, student_list):
    '''
    function to collect data from the simulation
    '''
    Name_list = []
    extraversion_list = []
    sensing_list = []
    thinking_list = []
    judging_list = []
    academic_list = []
    
    for student in studygroup:
        Name_list.append(student.Name)
        extraversion_list.append(student.ExScore)
        sensing_list.append(student.SeScore) 
        thinking_list.append(student.ThScore)
        judging_list.append(student.JuScore)
        academic_list.append(student.Academic_Skill)

    data = pd.DataFrame({'Name': Name_list, 
                        'type': student_list, 
                        'E/I': extraversion_list, 
                        'S/N': sensing_list,
                        'T/F': thinking_list,
                        'J/P': judging_list, 
                        'Academic': academic_list})
    
    return data
    

#### Create Study Group

In [36]:
## Defining the Students ##
class Student():
    def __init__(self, Name, Ex, Se, Th, Ju):
        self.Name  = Name

        ## Personality Traits ##
        self.Ex = Ex #Extraversion vs Introversion dimension
        self.Se = Se #Sensing vs Intuition dimension
        self.Th = Th #Thinking vs Feeling dimension
        self.Ju = Ju #Judging vs Perceiving dimension
        self.Type = Ex + Se + Th + Ju

        ## Personality Scores calculated with the personality() function##
        self.ExScore = 0
        self.SeScore = 0
        self.ThScore = 0
        self.JuScore = 0
        
        self.Scores = [] #list of all personality scores

        ## Academic Skills ##
        self.Academic_Skill = 0

        ## Own Solution ##
        self.Ind_Solution = []

def personality(student):
    # Extraversion vs. Introversion
    if student.Ex == "E":
        student.ExScore = max_vals[0]
        del max_vals[0]
    else:
        student.ExScore = min_vals[0]
        del min_vals[0]
    
    # Sensing vs. Intuition
    if student.Se == "S":
        student.SeScore = max_vals[0]
        del max_vals[0]
    else:
        student.SeScore = min_vals[0]
        del min_vals[0]
    
    # Thinking vs. Feeling
    if student.Th == "T":
        student.ThScore = max_vals[0]
        del max_vals[0]
    else:
        student.ThScore = min_vals[0]
        del min_vals[0]

    # Judging vs. Perceiving
    if student.Ju == "J":
        student.JuScore = max_vals[0]
        del max_vals[0]
    else:
        student.JuScore = min_vals[0]
        del min_vals[0]
    
    student.Scores = [student.ExScore, student.SeScore, student.ThScore, student.JuScore]

def skills(student):
    student.Academic_Skill = (1-student.SeScore)*0.33 + student.JuScore*0.33 + (rnd.beta(8, 2, 1)[0]*0.33)
    student.Compromising = (1-student.ThScore)*0.33 + student.ExScore*0.33 + (rnd.beta(8, 2, 1)[0]*0.33)

In [37]:
def StudyGroup(student_list):
    '''
    Create a study group of students
    '''
    studygroup = []
    names = ["Alfa", "Bravo", "Charlie", "Delta"]

    for i in range(len(student_list)):
        student = Student(names[i], student_list[i][0], student_list[i][1], student_list[i][2], student_list[i][3])
        personality(student)
        skills(student)
        studygroup.append(student)
    
    return data_collect(studygroup, student_list), studygroup

## Creating the ABM

### Generating True Solution

In [38]:
def true_solution_generator(n_part_exercises, range_elements):
    '''
    create a list of random numbers that will serve as the true solution that the agents need to find 
    '''
    true_solution = []
    for i in range(n_part_exercises):
        true_solution.append(random.randint(range_elements[0], range_elements[1]))

    return true_solution, range_elements

### Generating Individual Solutions

In [39]:
def individual_solutions_generator(studygroup, true_solution, range_elements):
    '''
    #function to calculate the individual solutions of the agents given a study group dataframe and a true solution
    '''
    
    all_solutions = []

    for student in studygroup[1]: # loop for each individual
        Ind_Solution_lst = []
        Ind_Solution_attr_lst = []
        
        for i in range(len(true_solution)): # loop for each part-exercise
            coin_toss = np.random.binomial(1, (student.Academic_Skill*0.5) + 0.5, 1)[0] # biased-coin flip
            if coin_toss == 1:
                Ind_Solution_lst.append(true_solution[i])
                Ind_Solution_attr_lst.append(true_solution[i])
            else:
                if student.Type[0] == "E":
                    wrong_answer = 1
                    Ind_Solution_lst.append(wrong_answer)
                    Ind_Solution_attr_lst.append(wrong_answer)
                if student.Type[0] == "I":
                    wrong_answer = 2
                    Ind_Solution_lst.append(wrong_answer)
                    Ind_Solution_attr_lst.append(wrong_answer)
        
        all_solutions.append(Ind_Solution_lst)
        student.Ind_Solution = Ind_Solution_attr_lst
    
    return all_solutions

In [40]:
def agent_type_equality(agent_a, agent_b):
    equality_score = 0 # number from 0 to 1
    for i in range(4):
        if agent_a[i] == agent_b[i]:
            equality_score += 0.25
    return equality_score 

In [41]:
A = "ESTJ"
B = "ISTJ"
agent_type_equality(A, B)

0.75

## Collaborative Problem Solving

##### Workflow notes
1. Who presents their solution e.g. agent A presents their solution to agent B, C, D -> THE PROPOSED SOLUTION
    -> Based on Extraversion score (Highest extraversion score is the most likely to present their solution)
2. According to an Agreeableness score (Social score for now) of the other agents (and maybe a Trustworthiness score of agent proposing), agents will update their solution
3. The solutions of the agents will be checked, if all they agree, this is their final solution. If not, the process will be repeated for a max of X ticks. If the groups do not converge, an accuracy score will still be calculated. 

In [42]:
def collaborative_solution(studygroup, max_ticks):
    max_ticks = max_ticks #turns in the simulation
    n_ticks = 0
    consensus = False
    
    # extracting all 4 students
    Alfa = studygroup[1][0]
    Bravo = studygroup[1][1]
    Charlie = studygroup[1][2]
    Delta = studygroup[1][3]

    student_list = [Alfa, Bravo, Charlie, Delta]
    individual_solutions = [Alfa.Ind_Solution, Bravo.Ind_Solution, Charlie.Ind_Solution, Delta.Ind_Solution]

    while (n_ticks != max_ticks):
        # Starting a Round
        presenter_name = random.choices([Alfa.Name, Bravo.Name, Charlie.Name, Delta.Name], weights = [Alfa.ExScore, Bravo.ExScore, Charlie.ExScore, Delta.ExScore], k = 1)[0] # selecting the presenter of the round based on weighted random draw from extraversion scores
        
        Proposed_Solution = eval(presenter_name).Ind_Solution
        #print(n_ticks, presenter_name, Proposed_Solution)

        for student in student_list:
            if student == eval(presenter_name):
                continue
            
            similarity_score = agent_type_equality(eval(presenter_name).Type, student.Type) # evaluate how equal the two agents are in personality. This bases the coinflip
            similarity_coin_toss = np.random.binomial(1, similarity_score, 1)[0]
            
            if similarity_coin_toss == 0:
                coin_toss = np.random.binomial(1, 0.5, 1)[0] # giving a 50% chance of not ending loop if you lose on similarity score.
                if coin_toss == 1:
                    continue
                
            for i in range(len(Proposed_Solution)): # looping through all part-exercises and evaluating against proposed solution.
                coin_toss = np.random.binomial(1, (student.Compromising*0.25 + eval(presenter_name).Academic_Skill*0.25), 1)[0] #high social skill has a greater chance of accepting the proposal and if the proposer has higher academic skill.
                if coin_toss == 1:
                    student.Ind_Solution[i] = Proposed_Solution[i]
                else:
                    student.Ind_Solution[i] = student.Ind_Solution[i]

        n_ticks += 1 #adding a tick to the simulation

        if all(x==individual_solutions[0] for x in individual_solutions): # initializes a break of while-loop if consensus has been reached (i.e. all solutions are the same)
            consensus = True
            break
    
    if consensus == True:
        final_group_solution = individual_solutions[0] # take one of the solutions from the group if they are all the same
    
    if consensus == False:
        final_group_solution = Proposed_Solution



    #print("Number of iterations: " + str(n_ticks))
    return final_group_solution, n_ticks, consensus # returns their collective solution


In [43]:
def solution_evaluator(proposed_solution, true_solution):
    correct_part_exercise = 0
    wrong_part_exercise = 0

    for i in range(len(true_solution)):
        if proposed_solution[i] == true_solution[i]:
            correct_part_exercise += 1
        else:
            wrong_part_exercise += 1

    return correct_part_exercise / (correct_part_exercise + wrong_part_exercise)


In [44]:
def adaptation_degree(individual_solution, group_solution):
    '''
    #function to calculate the adaptation degree of the agents
    '''
    adapted = 0
    not_adapted = 0

    for i in range(len(individual_solution)):
        if individual_solution[i] == group_solution[i]:
            not_adapted += 1
        else:
            adapted += 1
    
    adaptation = adapted / (adapted + not_adapted)

    return adaptation

In [45]:
def synergy_calculator(individual_accuracies, group_accuracy):
    '''
    #function to calculate the gain of the agents
    '''
    avg_of_indi_accuracy = np.mean(individual_accuracies)
    max_of_indi_accuracy = np.max(individual_accuracies)
    
    weak_synergy = group_accuracy - avg_of_indi_accuracy
    strong_synergy = group_accuracy - max_of_indi_accuracy

    return weak_synergy, strong_synergy

In [46]:
def diversity_score_calculator(student_list):
    diversity_score = 0

    for i in range(4):
        if (student_list[0][i] == student_list[1][i] == student_list[2][i] == student_list[3][i]):
            diversity_score += 0
        elif (student_list[0][i] == student_list[1][i] == student_list[2][i]) or (student_list[0][i] == student_list[1][i] == student_list[3][i]) or (student_list[1][i] == student_list[2][i] == student_list[3][i]) or (student_list[0][i] == student_list[2][i] == student_list[3][i]):
            diversity_score += 1
        else:
            diversity_score += 2
    
    return diversity_score

# TEST RUN

In [47]:
# Creating the Study Group
student_list_test = ["ESFP", "ISFP", "ISFP", "ISFP"]
studygroup1 = StudyGroup(student_list_test)

In [48]:
diversity_score_calculator(student_list_test)

1

In [49]:
# Seeing their Generated Personality Scores
print(studygroup1[0])

      Name  type       E/I       S/N       T/F       J/P  Academic
0     Alfa  ESFP  0.551934  0.617539  0.178827  0.301906  0.445872
1    Bravo  ISFP  0.129382  0.585067  0.329284  0.426310  0.531059
2  Charlie  ISFP  0.382529  0.825667  0.304898  0.306537  0.465312
3    Delta  ISFP  0.191405  0.536864  0.366709  0.383000  0.565813


In [50]:
# Generating a True Solution
true_solution_test = true_solution_generator(10, [1, 8])

In [51]:
# Displaying the True Solution
true_solution_test[0]

[1, 3, 2, 8, 3, 7, 1, 1, 1, 8]

In [52]:
# Generating Individual Solutions
all_solutions = individual_solutions_generator(studygroup1, true_solution_test[0], true_solution_test[1])
all_solutions

[[1, 3, 2, 1, 3, 1, 1, 1, 1, 8],
 [2, 3, 2, 2, 3, 7, 1, 1, 2, 2],
 [1, 2, 2, 8, 2, 7, 2, 1, 1, 8],
 [1, 2, 2, 2, 2, 2, 2, 1, 1, 2]]

In [53]:
# Printing Individual Solutions
for student in studygroup1[1]:
    print(student.Name, student.Ind_Solution)

Alfa [1, 3, 2, 1, 3, 1, 1, 1, 1, 8]
Bravo [2, 3, 2, 2, 3, 7, 1, 1, 2, 2]
Charlie [1, 2, 2, 8, 2, 7, 2, 1, 1, 8]
Delta [1, 2, 2, 2, 2, 2, 2, 1, 1, 2]


In [54]:
# Printing Indivudal Accuracies
for student in studygroup1[1]:
    print(student.Name, solution_evaluator(student.Ind_Solution, true_solution_test[0]))

Alfa 0.8
Bravo 0.6
Charlie 0.7
Delta 0.4


In [55]:
# Creating Group Solution
group_solution = collaborative_solution(studygroup1, 100)

In [56]:
print(group_solution)
print(all_solutions)

([1, 3, 2, 1, 3, 1, 2, 1, 2, 2], 21, True)
[[1, 3, 2, 1, 3, 1, 1, 1, 1, 8], [2, 3, 2, 2, 3, 7, 1, 1, 2, 2], [1, 2, 2, 8, 2, 7, 2, 1, 1, 8], [1, 2, 2, 2, 2, 2, 2, 1, 1, 2]]


In [57]:
# Printing Group Accuracy
accuracy = solution_evaluator(group_solution[0], true_solution_test[0])
print(accuracy)

0.5


In [58]:
adaptation_degree(all_solutions[0], group_solution[0])

0.3

## ABM Model

In [59]:
def exercise_run(student_list: list, max_ticks: int, n_part_exercises: int, range_solution: list, n_simulations: int):
    '''
    function to run one simulation of a study group completing the exercise (from individual to group solution)
    '''

    #### ---- LISTS ---- #### 
    ## ---- group level ---- ###
    group_accuracy_list = []
    n_tick_list = []
    weak_synergy_list = []
    strong_synergy_list = []
    consensus_list = []

    ## ---- individual level ---- ###
    alfa_accuracy_list = []
    beta_accuracy_list = []
    charlie_accuracy_list = []
    delta_accuracy_list = []

    alfa_adaptation_list = []
    beta_adaptation_list = []
    charlie_adaptation_list = []
    delta_adaptation_list = []

    # ---- starting simulation ---- #
    for i in range(n_simulations):
        ## giving the study group personality values and skills ##
        studygroup = StudyGroup(student_list)
        
        ## ---- SOLUTIONS ---- ##
        ## generating the true solution ## 
        true_solution = true_solution_generator(n_part_exercises, range_solution)

        ## generating individual solutions ##
        all_solutions = individual_solutions_generator(studygroup, true_solution[0], true_solution[1])

        ## evaluating own solutions ##        
        alfa_accuracy = solution_evaluator(all_solutions[0], true_solution[0])
        beta_accuracy = solution_evaluator(all_solutions[1], true_solution[0])
        charlie_accuracy = solution_evaluator(all_solutions[2], true_solution[0])
        delta_accuracy = solution_evaluator(all_solutions[3], true_solution[0])

        all_accuracies = [alfa_accuracy, beta_accuracy, charlie_accuracy, delta_accuracy]
        
        alfa_accuracy_list.append(alfa_accuracy)
        beta_accuracy_list.append(beta_accuracy)
        charlie_accuracy_list.append(charlie_accuracy)
        delta_accuracy_list.append(delta_accuracy)

        ## generating group solution ##
        group_solution = collaborative_solution(studygroup, max_ticks)
        n_tick_list.append(group_solution[1]) # appending the number of ticks
        consensus_list.append(group_solution[2]) # appending whether a consensus was reached or not
        
        ## ---- MEASURES ---- ##
        ## calculating degree of adaptation ## 
        alfa_adaption = adaptation_degree(all_solutions[0], group_solution[0])
        beta_adaption = adaptation_degree(all_solutions[1], group_solution[0])
        charlie_adaption = adaptation_degree(all_solutions[2], group_solution[0])
        delta_adaption = adaptation_degree(all_solutions[3], group_solution[0])

        alfa_adaptation_list.append(alfa_adaption)
        beta_adaptation_list.append(beta_adaption)
        charlie_adaptation_list.append(charlie_adaption)
        delta_adaptation_list.append(delta_adaption)

        ## calculating the accuracy of the group solution ##
        group_accuracy = solution_evaluator(group_solution[0], true_solution[0])
        group_accuracy_list.append(group_accuracy) #appending the group accuracies

        # calculating the gain from the individual solutions to the group solution
        weak_synergy, strong_synergy = synergy_calculator(all_accuracies, group_accuracy)
        weak_synergy_list.append(weak_synergy)
        strong_synergy_list.append(strong_synergy)
        

    
    diversity_score = diversity_score_calculator(student_list)

    ## calculating the mean and standard deivations ##
    # ---- group level ---- #
    avg_group_accuracy = np.mean(group_accuracy_list)
    std_group_accuracy = np.std(group_accuracy_list)
    
    avg_group_n_tick = np.mean(n_tick_list)
    std_group_n_tick = np.std(n_tick_list)

    avg_consensus = np.mean(consensus_list)
    std_consensus = np.std(consensus_list)

    avg_weak_synergy  = np.mean(weak_synergy_list)
    std_weak_synergy = np.std(weak_synergy_list)

    avg_strong_synergy  = np.mean(strong_synergy_list)
    std_strong_synergy = np.std(strong_synergy_list)
    
    # ---- individual level ---- #
    avg_alfa_accuracy = np.mean(alfa_accuracy_list)
    std_alfa_accuracy = np.std(alfa_accuracy_list)

    avg_beta_accuracy = np.mean(beta_accuracy_list)
    std_beta_accuracy = np.std(beta_accuracy_list)

    avg_charlie_accuracy = np.mean(charlie_accuracy_list)
    std_charlie_accuracy = np.std(charlie_accuracy_list)

    avg_delta_accuracy = np.mean(delta_accuracy_list)
    std_delta_accuracy = np.std(delta_accuracy_list)

    avg_alfa_adaptation = np.mean(alfa_adaptation_list)
    avg_beta_adaptation = np.mean(beta_adaptation_list)
    avg_charlie_adaptation = np.mean(charlie_adaptation_list)
    avg_delta_adaptation = np.mean(delta_adaptation_list)
    
    ## dictionaries ## 
    exercise_run_dict_group = {
    'alfa': student_list[0], 
    'beta':student_list[1], 
    'charlie':student_list[2], 
    'delta':student_list[3], 
    'avg_accuracy': avg_group_accuracy, 
    'std_accuracy': std_group_accuracy, 
    'avg_n_tick': avg_group_n_tick, 
    'std_n_tick': std_group_n_tick, 
    'avg_consensus': avg_consensus,
    'std_consensus': std_consensus,
    'avg_weak_synergy': avg_weak_synergy,
    'std_weak_synergy': std_weak_synergy, 
    'avg_strong_synergy': avg_strong_synergy,
    'std_strong_synergy': std_strong_synergy,
    'diversity_score': diversity_score}

    exercise_run_list_ind = list(zip(student_list, 
                                    [student_list, student_list, student_list, student_list], 
                                    [avg_alfa_accuracy, avg_beta_accuracy, avg_charlie_accuracy, avg_delta_accuracy],
                                    [std_alfa_accuracy, std_beta_accuracy, std_charlie_accuracy, std_delta_accuracy], 
                                    [avg_alfa_adaptation, avg_beta_adaptation, avg_charlie_adaptation, avg_delta_adaptation]))
    
    return exercise_run_dict_group, exercise_run_list_ind

In [60]:
def abm_model(condition, max_ticks: int, n_part_exercises: int, range_solution: list, n_simulations: int):
    condition_group_df = pd.DataFrame(columns = ['alfa', 'beta', 'charlie', 'delta', 'avg_accuracy', 'std_accuracy', 'avg_n_tick', 'std_n_tick', 'avg_consensus', 'std_consensus', 'avg_weak_synergy', 'std_weak_synergy', 'avg_strong_synergy', 'std_strong_synergy', 'diversity_score'])
    condition_ind_df = pd.DataFrame(columns=['type', 'studygroup', 'avg_accuracy', 'std_accuracy', 'avg_adaptation']) #'avg_accuracy', 'contribution', 'n_presentations'])
        
    for i in (range(len(condition.index))):
        student_list = list(condition.iloc[i][1:5])
        
        ## Getting data frames for the group and individual levels ##
        exercise_run_dict_group, exercise_run_list_ind = exercise_run(student_list, max_ticks, n_part_exercises, range_solution, n_simulations) #creating dictionary
        
        exercise_run_group_df = pd.DataFrame(exercise_run_dict_group, index = [i]) #creating dataframe
        condition_group_df = pd.concat([condition_group_df, exercise_run_group_df], ignore_index = True) #concatenating dataframes
        
        exercise_run_ind_df = pd.DataFrame(exercise_run_list_ind, columns=['type', 'studygroup', 'avg_accuracy', 'std_accuracy', 'avg_adaptation']) #'avg_accuracy', 'contribution', 'n_presentations'])
        condition_ind_df = pd.concat([condition_ind_df, exercise_run_ind_df], ignore_index = True)

    return condition_ind_df, condition_group_df


In [61]:
homogenous_ind_df, homogeous_group_df = abm_model(condition = homogenous, 
        max_ticks = 20, 
        n_part_exercises = 10, 
        range_solution = [1, 9], 
        n_simulations = 100
        )

In [62]:
sorted_wsynergy_homogenous = homogeous_group_df.sort_values("avg_accuracy", ascending = True)
sorted_wsynergy_homogenous

homogenous_ind_df.sort_values("avg_adaptation", ascending = True)

,type,studygroup,avg_accuracy,std_accuracy,avg_adaptation
30,INFJ,"[INFJ, INFJ, INFJ, INFJ]",0.899,0.096431,0.132
22,ENFJ,"[ENFJ, ENFJ, ENFJ, ENFJ]",0.879,0.106108,0.133
37,ENTJ,"[ENTJ, ENTJ, ENTJ, ENTJ]",0.885,0.109886,0.138
29,INFJ,"[INFJ, INFJ, INFJ, INFJ]",0.878,0.113649,0.139
28,INFJ,"[INFJ, INFJ, INFJ, INFJ]",0.883,0.103976,0.146
...,...,...,...,...,...
55,ESTP,"[ESTP, ESTP, ESTP, ESTP]",0.742,0.135041,0.267
52,ESTP,"[ESTP, ESTP, ESTP, ESTP]",0.783,0.129657,0.268
54,ESTP,"[ESTP, ESTP, ESTP, ESTP]",0.757,0.147821,0.270
60,ISTP,"[ISTP, ISTP, ISTP, ISTP]",0.782,0.129908,0.278


In [63]:
fifty_ind, fifty_group = abm_model(condition = fiftyfifty, 
        max_ticks = 20, 
        n_part_exercises = 10, 
        range_solution = [1, 9], 
        n_simulations = 100
        )

In [64]:
sorted_wsynergy_fifty = fifty_group.sort_values("avg_accuracy", ascending = True)
sorted_wsynergy_fifty

,alfa,beta,charlie,delta,avg_accuracy,std_accuracy,avg_n_tick,std_n_tick,avg_consensus,std_consensus,avg_weak_synergy,std_weak_synergy,avg_strong_synergy,std_strong_synergy,diversity_score
10,ESFP,ESFP,INTP,INTP,0.778,0.127734,16.52,4.215400,0.55,0.497494,-1.750000e-03,0.108585,-0.140,0.124097,6
6,ESTP,ESTP,INFJ,INFJ,0.779,0.130610,17.11,3.929109,0.51,0.499900,-3.750000e-02,0.111664,-0.174,0.134625,8
14,ESTP,ESTP,ISFJ,ISFJ,0.781,0.124656,15.65,4.879293,0.62,0.485386,-4.500000e-03,0.117334,-0.133,0.124944,6
8,ESFJ,ESFJ,ISTP,ISTP,0.800,0.117473,15.26,5.007235,0.63,0.482804,1.300000e-02,0.101887,-0.119,0.109266,6
4,ESTJ,ESTJ,INFP,INFP,0.800,0.123288,15.09,4.917510,0.66,0.473709,-7.250000e-03,0.109390,-0.123,0.119042,8
12,ESTJ,ESTJ,ISFP,ISFP,0.810,0.126886,15.33,4.662735,0.68,0.466476,1.700000e-02,0.107464,-0.110,0.117898,6
2,ESFP,ESFP,INTJ,INTJ,0.811,0.134086,16.10,5.193265,0.51,0.499900,-8.750000e-03,0.113047,-0.142,0.133551,8
0,ESFJ,ESFJ,INTP,INTP,0.815,0.121963,16.65,4.355169,0.52,0.499600,2.250000e-03,0.108921,-0.113,0.122192,8
1,ENTP,ENTP,ISFJ,ISFJ,0.820,0.118322,15.61,5.308286,0.59,0.491833,8.881784e-18,0.097468,-0.118,0.122784,8
5,ENFP,ENFP,ISTJ,ISTJ,0.822,0.116258,16.08,4.601478,0.57,0.495076,-3.250000e-03,0.090599,-0.120,0.105830,8


In [65]:
hetero_ind, hetero_group = abm_model(condition = heterogenous, 
        max_ticks = 20, 
        n_part_exercises = 10, 
        range_solution = [1, 9], 
        n_simulations = 100
        )

In [66]:
sorted_wsynergy_hetero = hetero_group.sort_values("avg_accuracy", ascending = True)
sorted_wsynergy_hetero

,alfa,beta,charlie,delta,avg_accuracy,std_accuracy,avg_n_tick,std_n_tick,avg_consensus,std_consensus,avg_weak_synergy,std_weak_synergy,avg_strong_synergy,std_strong_synergy,diversity_score
8,ESFJ,ISTP,ESTP,ENTP,0.779,0.129070,15.30,4.620606,0.70,0.458258,-0.01450,0.107888,-0.145,0.125200,4
13,ENTP,INTP,ESFP,ISFP,0.781,0.118908,15.96,4.235375,0.64,0.480000,-0.00425,0.111246,-0.136,0.130015,6
6,ENFP,ISFP,ESFP,INFP,0.790,0.126095,13.44,5.076061,0.83,0.375633,0.00150,0.106055,-0.139,0.117384,4
9,ESTJ,ISFP,ISTJ,ESFP,0.806,0.106602,14.89,4.870103,0.69,0.462493,0.01950,0.098080,-0.109,0.108715,6
5,ESTJ,ESTP,ISTP,INTJ,0.807,0.140182,15.18,4.838140,0.67,0.470213,-0.00825,0.114894,-0.137,0.137590,5
12,ESFJ,ISFJ,ENFP,INFP,0.817,0.119210,14.15,5.105634,0.75,0.433013,-0.00375,0.103825,-0.127,0.115633,6
2,ISTJ,ISTP,ESFP,ENTJ,0.820,0.137840,15.32,4.837107,0.65,0.476970,0.01500,0.125847,-0.116,0.137637,6
1,ISFJ,ESTP,ISFP,ENFJ,0.822,0.126949,14.52,4.998960,0.72,0.448999,0.01175,0.102558,-0.118,0.115222,6
3,INTP,INTJ,INFP,ENTP,0.827,0.119042,13.42,5.301283,0.81,0.392301,-0.01100,0.101077,-0.125,0.100374,3
4,ESFJ,ISFJ,ISTJ,INTP,0.827,0.124784,14.56,5.020598,0.73,0.443959,-0.00625,0.103705,-0.113,0.123008,5


In [67]:
homogenous_mean_acc = np.mean(homogeous_group_df['avg_accuracy'])
fifty_mean_acc = np.mean(fifty_group['avg_accuracy'])
hetero_mean_acc = np.mean(hetero_group['avg_accuracy'])

homogenous_std_acc = np.std(homogeous_group_df['avg_accuracy'])
fifty_std_acc = np.std(fifty_group['avg_accuracy'])
hetero_std_acc = np.std(hetero_group['avg_accuracy'])

homogenous_mean_tick = np.mean(homogeous_group_df['avg_n_tick'])
fifty_mean_tick = np.mean(fifty_group['avg_n_tick'])
hetero_mean_tick = np.mean(hetero_group['avg_n_tick'])

homogenous_std_tick = np.std(homogeous_group_df['avg_n_tick'])
fifty_std_tick = np.std(fifty_group['avg_n_tick'])
hetero_std_tick = np.std(hetero_group['avg_n_tick'])

print(tuple([homogenous_mean_acc, fifty_mean_acc, hetero_mean_acc]))
print(tuple([homogenous_std_acc, fifty_std_acc, hetero_std_acc]))

print(tuple([homogenous_mean_tick, fifty_mean_tick, hetero_mean_tick]))
print(tuple([homogenous_std_tick, fifty_std_tick, hetero_std_tick]))

(0.8249375000000001, 0.8216875000000001, 0.8210000000000002)
(0.043674175364281365, 0.030864054233849456, 0.023229829960634746)
(10.788124999999999, 14.872500000000002, 13.901874999999999)
(1.9839267210194536, 1.8696039019000787, 1.3748418943191252)
